# LAB 6 : Feature selection methods
All the different approaches to feature selection can be grouped into mainly two families of methods. There are unsupervised and supervised methods. The latter can be further divided into the wrapper and filter mothods. Let’s discuss them one by one.

In this lab, your task is to carefully review all instructions and fill the empty code cells with the necessary code to ensure everything functions correctly.


## Import necessary libraries

Requirement:
- pip install mlxtend
- pip install sklearn-genetic-opt

In [ ]:
!pip install mlxtend
!pip install sklearn-genetic-opt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 5.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import warnings

from sklearn.datasets import load_iris, load_breast_cancer, fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold, SelectKBest, chi2, mutual_info_classif, f_classif
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn_genetic import GAFeatureSelectionCV
warnings.filterwarnings("ignore")


# About the dataset



**The Forest Cover Type Dataset (Covertype)** contains data on forest cover types based on cartographic variables.

It is composed of 110,393 instances and 54 features (including continuous, categorical, and binary features) that represent various environmental factors, making it ideal for exploring feature selection.

The classification task is to predict the type of forest cover (seven possible types).

Here is a description of each forest cover class in the used dataset:

**Spruce/Fir (Class 1):** A forest type primarily composed of spruce and fir trees, which are common in colder climates and higher elevations. These trees are known for their needle-like leaves and are typically evergreen.

**Lodgepole Pine (Class 2):** Dominated by lodgepole pine trees, which grow well in diverse soil types and are highly adaptable. Lodgepole pines are resilient, often thriving in areas prone to wildfires as their cones require heat to open and release seeds.

**Ponderosa Pine (Class 3):** Ponderosa pine forests are found at lower elevations and warmer climates. These trees are tall with thick bark and long needles, often providing valuable habitats and resources for wildlife.

**Cottonwood/Willow (Class 4):** A mix of cottonwood and willow trees, often found near water sources like rivers or lakes due to their preference for moist soils. These trees are deciduous, shedding leaves seasonally, and play an essential role in riparian ecosystems.

**Aspen (Class 5):** Aspen forests consist mainly of aspen trees, known for their smooth, white bark and leaves that tremble in the wind. Aspens grow in clonal colonies and are usually found in cooler, moist environments.

**Douglas-fir (Class 6):** A forest type dominated by Douglas-fir trees, which are common in the western U.S. These tall trees with conical shapes are important for the timber industry and thrive in both dry and wet climates.

**Krummholz (Class 7):** Known as "twisted wood," Krummholz forests are made up of trees that are stunted and gnarled due to harsh environmental conditions, such as strong winds and cold temperatures. They are commonly found at the treeline in mountainous areas.

In [ ]:
# Load the dataset
data = fetch_openml("Covertype", version=1)

initial_data = pd.DataFrame(data.data)
initial_data['class'] = data.target

initial_data.dtypes

#select Continuous and Categorical features
continuous_features = initial_data.#FIX ME#
categorical_features = initial_data.#FIX ME# Careful not to include target feature

#Print their shape
#Fix ME#

SyntaxError: invalid syntax (ipython-input-2789073567.py, line 10)

# Data sampling

For learning purposes, and due to the large size of the Covertype dataset, using the entire dataset during the lab session would be impractical. Instead, we will work with a sample to make our analysis and modeling more manageable. To ensure that our sample reflects the original distribution of the target variable, we will perform **stratified sampling**. This approach will allow us to preserve the relative proportions of each forest cover type within our sample, ensuring that our insights and model results remain representative of the full dataset.

In [ ]:
# Create a 1% stratified sample from the original dataset ensuring reproducibility
X_sampled, _ = train_test_split(initial_data, #FIX ME#, #FIX ME#, #FIX ME#)
print(X_sampled.shape)

Display a Bar chart to visualize the class distribution in the dataset before and after performing the stratified sampling

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
initial_data['class'].value_counts().plot(kind='bar', color='#4287f5')
plt.title('Original Class Distribution')
plt.xlabel('Class')
plt.ylabel('Frequency')

# Sampled class distribution
plt.subplot(1, 2, 2)
X_sampled['class'].value_counts().plot(kind='bar', color='#FF5733')
plt.title('Sampled Class Distribution')
plt.xlabel('Class')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
#Splitting the sampled dataset into features and target
X = X_sampled.iloc[:,:-1]
y = pd.Series(X_sampled['class'], name='target')
print(f"Number of Features : {X.shape[1]}")
print(f"Sample of Featues values : \n {X.head}")
print("#############################")
print(f"Sample of Target values : \n {y.head}")

## Split the dataset into training and testing sets

**We need to explain what is spliting and why do we need it (model evaluation):**

In this classification problem, our goal is to develop a model that can accurately classify flowers based on given features. To achieve this, we need to follow a few critical steps:

(1) *Data Preprocessing* (already done).

(2) **Splitting the Dataset:** We divide the dataset into two subsets—the Training Set and the Testing Set.

(3) **Model Training:** Using the training set, we provide the model with pairs of features and their respective class labels, enabling it to learn patterns in the data and understand when to assign certain classes to specific examples (rows).

(4) **Model Testing:** Once training is complete, we evaluate the model by testing it on the testing set. This assessment helps us determine how well the model generalizes to new, unseen data (i.e., can the model correctly classify flowers it hasn’t encountered during training?).

### **Why Splitting Is Necessary**

Splitting the data is crucial for reliable model evaluation. By keeping part of the data aside for testing, we can better assess the model's ability to generalize to new data, which reflects real-world scenarios. If we were to train and test on the same data, the model might perform well simply because it has **"memorized"** the data, rather than learned **generalizable patterns**. This could lead to overfitting, where the model fails on unseen data.

**Note: For this lab, we’ll use accuracy as the performance metric. However, there are several other metrics that can provide a more comprehensive evaluation of a model’s performance, especially in cases with imbalanced classes.**

In [ ]:
#Split the data set into two subsets, 80% for training and 20% for testing.
X_train, X_test, y_train, y_test = train_test_split(
                                                    X,
                                                    y,
                                                    #FIX ME#,
                                                    #FIX ME#
                                                    )

print(f"Training set's shape : {X_train.shape}")
print(f"Targets training set's shape : {y_train.shape}")
print("####################################################")
print(f"Testing set's shape : {X_test.shape}")
print(f"Targets testing set's shape : {y_test.shape}")

Now, to the Feature selection methods, as mentioned before, two main categories of methods could be distinguished: Unsupervised and Supervised

# Unsupervised feature selection methods


In this section we will see three different methods; **Variance threshold**, **Correlation** and **Mutual information**

## Variance Threshold

In [ ]:
def variance_threshold_selector(X, threshold=0.0):
    selector = VarianceThreshold(#FIX ME#)
    selector.fit(X)
    return X[X.columns[selector.get_support(indices=True)]]

In [ ]:
print(f"Initial features: \n{X.columns.shape}, {X.columns.values}")
X_var = variance_threshold_selector(
    #FIX ME#,
    threshold=0.1
    )
print(f"Selected features Shape and names: \n{X_var.columns.shape}, {X_var.columns.values}")

## Correlation

In [ ]:
cor = #FIX ME#.corr().abs()
plt.figure(figsize=(12, 8))
sns.heatmap(cor, annot=False, cmap=plt.cm.Reds)
plt.show()

In [ ]:
def correlation_selector(X, threshold=0.8):
    corr_matrix = #FIX ME#.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return X.drop(to_drop, axis=1)

In [ ]:
print(f"Initial features: \n{X.columns.values}")
#Set timer start
start_time = #FIX ME#
X_corr = correlation_selector(X_train[continuous_features], threshold=0.7)
#Set timer end
end_time = #FIX ME#
print(f"Selected features: \n{X_corr.columns.shape}, {X_corr.columns.values}")
print(f"Execution time: \n{end_time - start_time}")

## Mutual Information

Mutual information of two attributes (features or variables) A and B is the amount of information shared between them. Or how much knowing A reduces the uncertainty (Entropy) about knowing B.

Formula: $$ MI(A,B) = H(A) - H(A|B) $$

In [ ]:
# Feature selection using mutual information
def mutual_info_selector(X, y, top_k=4):
    selector = SelectKBest(
        #FIX ME#,
        k=#FIX ME#)
    selector.fit(X, y)
    return X[X.columns[selector.get_support(indices=True)]]

In [ ]:
# Feature selection using mutual information
print(f"Initial features: \n{X.columns.values}")
#Set timer start
start_time = #FIX ME#
X_mi = mutual_info_selector(
    #FIX ME#,
    #FIX ME#,
    top_k=20)
#Set timer end
end_time = #FIX ME#
print(f"Selected features: \n{X_mi.columns.values}")
print(f"Execution time: \n{end_time - start_time}")


# Supervised feature selection methods

## Information Gain

In [ ]:
# Feature selection using filter methods: information gain
def information_gain_selector(X, y, top_k=4):
    selector = SelectKBest(#FIX ME#, k=top_k)
    selector.fit(X, y)
    return X[X.columns[selector.get_support(indices=True)]]

In [ ]:
# Feature selection using information gain
print(f"Initial features: \n{X.columns.values}")
#Set timer start
start_time = #FIX ME#
X_ig = information_gain_selector(X_train[categorical_features], y_train, top_k=20)
#Set timer start
end_time = #FIX ME#
print(f"Selected features: \n{X_ig.columns.values}")
print(f"Execution time: \n{end_time - start_time}")

## Chi-squared Test

In [ ]:
def chi_squared_selector(X, y, top_k=4):
    selector = SelectKBest(#FIX ME#, k=top_k)
    selector.fit(X, y)
    return X[X.columns[selector.get_support(indices=True)]]

In [ ]:
# Feature selection using chi-squared test
print(X.columns)
X_chi2 = chi_squared_selector(#FIX ME#, top_k=20)
print(X_chi2.columns)

In [ ]:
# Create a random forest classifier (you can replace this with any other classifier of your choice)
model = RandomForestClassifier(n_estimators=50, random_state=42, criterion="entropy")

## Feature Selection Using Wrapper Methods:

## Forward Selection

In [ ]:
# Feature selection using wrapper methods: forward selection
def forward_selection(X, y, model):
    sfs = SequentialFeatureSelector(
                                    #FIX ME#,
                                    k_features='best',
                                    #FIX ME#,
                                    #FIX ME#,
                                    cv=2)
    sfs.fit(X, y)
    return X[X.columns[list(sfs.k_feature_idx_)]]

In [ ]:
# Feature selection using forward selection
print(f"Initial features: \n{X.columns.values}")
start_time =#FIX ME#
X_forward = forward_selection(X_train, y_train, model)
end_time = #FIX ME#
print(f"Selected features: \n{X_forward.columns.shape}, {X_forward.columns.values}")
print(f"Execution time: \n{end_time - start_time}")

## Backward Elimination

In [ ]:
def backward_elimination(X, y, model):
    sfs = SequentialFeatureSelector(
        #FIX ME#,
        k_features='best',
        #FIX ME#,
        #FIX ME#,
        cv=2)
    sfs.fit(X, y)
    return X[X.columns[list(sfs.k_feature_idx_)]]

In [ ]:
# Feature selection using backward elimination
print(f"Initial features: \n{X.columns.values}")
start_time = #FIX ME#
X_backward = #FIX ME#
end_time = #FIX ME#
print(f"Selected features: \n{X_backward.columns.shape}, {X_backward.columns.values}")
print(f"Execution time: \n{end_time - start_time}")

## Genetic Algorithm

In [ ]:
def genetic_algorithm_selector(X, y, model):
    selector = GAFeatureSelectionCV(model,
                                         cv=2,
                                         scoring='accuracy',
                                         population_size=10,
                                         generations=5,
                                         tournament_size=5,
                                         elitism=True,
                                         crossover_probability=0.9,
                                         mutation_probability=0.1,
                                         criteria='max',
                                         algorithm='eaMuPlusLambda',
                                         n_jobs=3,
                                         verbose=True,
                                         keep_top_k=4)

    selector.fit(X, y)
    return X[X.columns[selector.support_]]

In [ ]:
print(f"Initial features: \n{X.columns.values}")
start_time = time.time()
X_genetic = genetic_algorithm_selector(X_train, y_train, model)
end_time = time.time()
print(f"Selected features: \n{X_genetic.columns.shape}, {X_genetic.columns.values}")
print(f"Execution time: \n{end_time - start_time}")

## --Evaluate Model performance after feature selection--

## evaluate model Function

This function aims to evaluate a given model using Training and testing subsets. It is worth noting that we can also choose other metrics to evaluate the model's performance (comparison between predicted and true values)

In [ ]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train) #Train the model using training sets
    y_pred = model.predict(X_test)#Predict using testing set
    accuracy = accuracy_score(y_test, y_pred)#Using Accuracy metric, Compare the predicted values with the testing target values
    return accuracy

## Compare model performance considering the selected features

In [ ]:

# Concatenate the selected features of each method with the remaining non treated features (of the originial dataset)
X_variance = pd.concat(#FIX ME#)
X_correlation = pd.concat(#FIX ME#)
X_mInformation = pd.concat(#FIX ME#)
X_iGain = pd.concat(#FIX ME#)
X_chiSquare = pd.concat(#FIX ME#)

selected_features = [#FIX ME#]

selected_feature_names = ['Original', 'Variance Threshold', 'Correlation', 'Mutual Information', 'Information Gain', 'Chi-squared Test', 'Forward Selection', 'Backward Elimination', 'Genetic Algorithm']

results = []

for name, X_selected in zip(selected_feature_names, selected_features):
    X_test_selected = #FIX ME#  # Apply the same feature selection to the test set
    accuracy = evaluate_model(#FIX ME#)
    results.append({'Method': name, 'Accuracy': accuracy})

In [ ]:
# Display the results
results_df = pd.DataFrame(results)
print(results_df)

In [ ]:
# Plot the results
plt.figure(figsize=(10, 6))
sns.barplot(x='Accuracy', y='Method', data=results_df.sort_values(by='Accuracy', ascending=False))
plt.title('Accuracy after Feature Selection')
plt.show()

## **Q/A:** What do you notice ?

## Built in features importance  

In [ ]:
# Train the model on the whole feature set
model.fit(X_train, y_train)

importances = #FIX ME# # Get importance features from RandomForest model (search for it)
feature_imp_df = pd.DataFrame(
                              {'Feature': #FIX ME#,
                               'Importance': #FIX ME#
                               }).sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 14))
plt.barh(X_train.columns.values, importances, color='skyblue')
plt.xlabel('Importance')
plt.title('Feature Importance - Entropy Importance')
plt.gca().invert_yaxis()  # Invert y-axis for better visualization
plt.show()


## Hybrid Feature selection

Think of a way to combine two feature selection methods, Perform it, and show the corresponding results.

In [ ]:
# FIX ME


## Extra task

Explore the application of PCA on the original sampled dataset;

(1) Make sure 90% of variance is maintained.

(2) Choose 2 principale components and plot the reduced data.
